<a href="https://colab.research.google.com/github/Pratyush-3003/CLTV/blob/main/GUI.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [7]:
!pip install streamlit
!pip install lifetimes
import streamlit as st
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from lifetimes import BetaGeoFitter, GammaGammaFitter
from lifetimes.utils import summary_data_from_transaction_data

st.set_page_config(page_title="CLV Predictor", layout="wide")
st.title("📊 Customer Lifetime Value (CLV) Predictor")

uploaded_file = st.file_uploader("Upload transaction CSV file", type=["csv"])

if uploaded_file:
    df = pd.read_csv(uploaded_file, encoding='latin-1')

    # Preprocessing
    df.dropna(inplace=True)
    df.drop_duplicates(inplace=True)
    df = df[df['Quantity'] > 0]
    df = df[df['UnitPrice'] > 0]
    df['InvoiceDate'] = pd.to_datetime(df['InvoiceDate'], errors='coerce')
    df['CustomerID'] = df['CustomerID'].astype(int)
    df['TotalPrice'] = df['Quantity'] * df['UnitPrice']

    summary = summary_data_from_transaction_data(df, 'CustomerID', 'InvoiceDate', 'TotalPrice')

    # Modeling
    bgf = BetaGeoFitter(penalizer_coef=0.1)
    bgf.fit(summary['frequency'], summary['recency'], summary['T'])

    ggf = GammaGammaFitter(penalizer_coef=0)
    ggf.fit(summary['frequency'][summary['frequency'] > 0], summary['monetary_value'][summary['frequency'] > 0])

    summary['predicted_purchases'] = bgf.conditional_expected_number_of_purchases_up_to_time(
        90, summary['frequency'], summary['recency'], summary['T']).round().astype(int)

    summary['predicted_monetary'] = ggf.conditional_expected_average_profit(
        summary['frequency'], summary['monetary_value'])

    summary['CLV'] = ggf.customer_lifetime_value(
        bgf, summary['frequency'], summary['recency'],
        summary['T'], summary['monetary_value'], time=24, discount_rate=0.00
    )

    st.subheader("CLV Predictions Table")
    st.write(summary[['predicted_purchases', 'predicted_monetary', 'CLV']].head())

    # Plot
    clv_filtered = summary[summary['CLV'] < 10000]['CLV']
    p25, p50, p75 = np.percentile(clv_filtered, [25, 50, 75])

    fig, ax = plt.subplots(figsize=(10, 6))
    sns.histplot(clv_filtered, bins=50, kde=True, color='skyblue')
    plt.axvline(p25, color='green', linestyle='--', label=f'25th %ile: {p25:.0f}')
    plt.axvline(p50, color='blue', linestyle='--', label=f'50th %ile: {p50:.0f} (Median)')
    plt.axvline(p75, color='red', linestyle='--', label=f'75th %ile: {p75:.0f}')
    plt.legend()
    ax.set_title("CLV Distribution")
    st.pyplot(fig)


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 584.2/584.2 kB 9.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.4/119.4 kB 10.2 MB/s eta 0:00:00


2025-04-10 18:49:25.829 WARNING streamlit.runtime.scriptrunner_utils.script_run_context: Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-04-10 18:49:25.830 WARNING streamlit.runtime.scriptrunner_utils.script_run_context: Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-04-10 18:49:25.906 
  command:

    streamlit run /usr/local/lib/python3.11/dist-packages/colab_kernel_launcher.py [ARGUMENTS]
2025-04-10 18:49:25.907 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-04-10 18:49:25.909 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-04-10 18:49:25.910 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-04-10 18:49:25.912 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when runn